# Baran Experiment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import IPython.display

import raha
import datetime
from ruska import Ruska, Inspector

# Pdep auf Bridges

In [6]:
c = {'dataset': 'bridges',
     'sampling': 'MCAR',
     'error_fraction': .04,
     'labeling_budget': 20,
     'feature_generators': ['value', 'domain', 'vicinity'],
     'classification_model': 'ABC',
     'vicinity_orders': [1],
     'vicinity_feature_generator': 'naive',
     'n_best_pdeps': 5,
     'score_strategy': 'multiply',
     'n_rows': None,
     }

rate_formatted = int(str(c['error_fraction']).split(".")[1])
run = 1
data_dict = {
    "name": c["dataset"],
    "path": f"../datasets/renuver/{c['dataset']}/{c['dataset']}_{rate_formatted}_{run}.csv",
    "clean_path": f"../datasets/renuver/{c['dataset']}/clean.csv",
}

data = raha.Dataset(data_dict, n_rows=c["n_rows"])
data.detected_cells = dict(data.get_actual_errors_dictionary())
app = raha.Correction()
app.LABELING_BUDGET = c["labeling_budget"]
app.VERBOSE = False
app.FEATURE_GENERATORS = c["feature_generators"]
app.CLASSIFICATION_MODEL = c["classification_model"]
app.VICINITY_ORDERS = c["vicinity_orders"]
app.VICINITY_FEATURE_GENERATOR = c["vicinity_feature_generator"]
app.N_BEST_PDEPS = c["n_best_pdeps"]
app.PDEP_SCORE_STRATEGY = c["score_strategy"]

d_naive = app.initialize_dataset(data)
app.initialize_models(d_naive)
while len(d_naive.labeled_tuples) < app.LABELING_BUDGET:
    app.sample_tuple(d_naive, random_seed=None)
    app.label_with_ground_truth(d_naive)
    app.update_models(d_naive)
    app.generate_features(d_naive, synchronous=True)
    app.predict_corrections(d_naive, random_seed=None)

    p, r, f = d_naive.get_data_cleaning_evaluation(d_naive.corrected_cells)[-3:]
print(f'Label {len(d_naive.labeled_tuples)}: Cleaning F1-Score {round(f, 3)}.')

Label 20: Cleaning F1-Score 0.839.


In [5]:
c = {'dataset': 'bridges',
     'sampling': 'MCAR',
     'error_fraction': .04,
     'labeling_budget': 20,
     'feature_generators': ['value', 'domain', 'vicinity'],
     'classification_model': 'ABC',
     'vicinity_orders': [1,2],
     'vicinity_feature_generator': 'pdep',
     'n_best_pdeps': 5,
     'score_strategy': 'multiply',
     'n_rows': None,
     }

rate_formatted = int(str(c['error_fraction']).split(".")[1])
run = 1
data_dict = {
    "name": c["dataset"],
    "path": f"../datasets/renuver/{c['dataset']}/{c['dataset']}_{rate_formatted}_{run}.csv",
    "clean_path": f"../datasets/renuver/{c['dataset']}/clean.csv",
}

data = raha.Dataset(data_dict, n_rows=c["n_rows"])
data.detected_cells = dict(data.get_actual_errors_dictionary())
app = raha.Correction()
app.LABELING_BUDGET = c["labeling_budget"]
app.VERBOSE = False
app.FEATURE_GENERATORS = c["feature_generators"]
app.CLASSIFICATION_MODEL = c["classification_model"]
app.VICINITY_ORDERS = c["vicinity_orders"]
app.VICINITY_FEATURE_GENERATOR = c["vicinity_feature_generator"]
app.N_BEST_PDEPS = c["n_best_pdeps"]
app.PDEP_SCORE_STRATEGY = c["score_strategy"]

d_pdep = app.initialize_dataset(data)
app.initialize_models(d_pdep)
while len(d_pdep.labeled_tuples) < app.LABELING_BUDGET:
    app.sample_tuple(d_pdep, random_seed=None)
    app.label_with_ground_truth(d_pdep)
    app.update_models(d_pdep)
    app.generate_features(d_pdep, synchronous=True)
    app.predict_corrections(d_pdep, random_seed=None)

    p, r, f = d_pdep.get_data_cleaning_evaluation(d_pdep.corrected_cells)[-3:]
print(f'Label {len(d_pdep.labeled_tuples)}: Cleaning F1-Score {round(f, 3)}.')

Label 20: Cleaning F1-Score 0.786.


In [26]:
d.create_repaired_dataset(d.corrected_cells)
df_treated = d.repaired_dataframe
df_dirty = d.dataframe
df_clean = d.clean_dataframe

## Spalte 2 `Location`

In [33]:
ins = Inspector(assume_errors_known=True)
ins.calculate_cleaning_error_positions(df_clean.iloc[:, 2], df_dirty.iloc[:, 2])

In [62]:
ins.inspect_cleaning_results(df_clean, df_treated, df_dirty, slice(0,13), context_height=2)

Evaluating error 0 from 108
Error in row 3:
   ID RIVER LOCATION ERECTED   PURPOSE LENGTH LANES CLEAR-G   T-OR-D MATERIAL  \
1  E2     A       25    1819   HIGHWAY   1037     2       N  THROUGH     WOOD   
2  E3     A       39    1829  AQUEDUCT      ?     1       N  THROUGH     WOOD   
3  E5     A        ?    1837   HIGHWAY   1000     2       N  THROUGH     WOOD   
4  E6     M       23    1838   HIGHWAY      ?     2       N  THROUGH     WOOD   

    SPAN REL-L  TYPE  
1  SHORT     S  WOOD  
2      ?     S  WOOD  
3  SHORT     S  WOOD  
4      ?     S  WOOD  
Cleaning result in row 3:
   ID RIVER LOCATION ERECTED   PURPOSE LENGTH LANES CLEAR-G   T-OR-D MATERIAL  \
1  E2     A       25    1819   HIGHWAY   1037     2       N  THROUGH     WOOD   
2  E3     A       39    1829  AQUEDUCT      ?     1       N  THROUGH     WOOD   
3  E5     A       29    1837   HIGHWAY   1000     2       N  THROUGH     WOOD   
4  E6     M       23    1838   HIGHWAY      ?     2       N  THROUGH     WOOD   

   